In [1]:
import timm
import torch

In [2]:
model = timm.create_model("vit_base_resnet50_384", pretrained=True)
# model = timm.create_model("resnet50", pretrained=True)

In [3]:
x = torch.rand((4,3,384,384)).to('cuda')

In [23]:
EMBED_DIM = 768
n_acts = 3
embed = torch.nn.Embedding(n_acts + 1, EMBED_DIM)

In [24]:
model.embed = embed
model.to('cuda')

VisionTransformer(
  (patch_embed): HybridEmbed(
    (backbone): ResNetV2(
      (stem): Sequential(
        (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
        (norm): GroupNormAct(
          32, 64, eps=1e-05, affine=True
          (act): ReLU(inplace=True)
        )
        (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
      )
      (stages): Sequential(
        (0): ResNetStage(
          (blocks): Sequential(
            (0): Bottleneck(
              (downsample): DownsampleConv(
                (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (norm): GroupNormAct(
                  32, 256, eps=1e-05, affine=True
                  (act): Identity()
                )
              )
              (conv1): StdConv2dSame(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm1): GroupNormAct(
                32, 64, eps=1e-05, 

In [29]:
import types
def forward_features(self, x, actions):
    x = self.patch_embed(x)
    print(x.shape)
    
    cls_token = self.cls_token.expand(x.shape[0], -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
    
    act_embed = self.embed(actions).reshape(x.shape[0], 1, 768)
    cls_token = act_embed
    
    if self.dist_token is None:
        x = torch.cat((cls_token, x), dim=1)
    else:
        x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)
    x = self.pos_drop(x + self.pos_embed)
    x = self.blocks(x)
    x = self.norm(x)
    if self.dist_token is None:
        return self.pre_logits(x[:, 0])
    else:
        return x[:, 0], x[:, 1]
    
model.forward_features = types.MethodType(forward_features, model) 

In [30]:
actions = torch.tensor([1,2,3,1]).to('cuda')
model.forward_features(x, actions).shape

torch.Size([4, 576, 768])
torch.Size([4, 1, 768])
torch.Size([4, 1, 768])


torch.Size([4, 768])

In [ ]:
timm.list_models(pretrained=True)